# 통계청 '온라인 가격 정보' 오픈 API 수집 방법
$$ $$
* 통계청에서 2015년부터 온라인 가격 정보를 수집하여 오픈 API로 제공하고 있습니다.
$$ $$
* [공공데이터 포털](http://www.data.go.kr)에서 인증키를 발급받으시면 됩니다.
  - 인증키 발급방법은 ['공공데이터포털 OpenAPI 인증키 발급방법'](http://nbviewer.jupyter.org/format/slides/gist/KevinSHNa/89aabf84f7e8eae51a334c87a624e31f#/)를 참고하시기 바랍니다.
$$ $$
* 온라인 가격 정보는 품목조회와 가격조회로 구성되어 있습니다. 
  - 먼저 품목조회에서 원하는 품목 코드를 찾습니다. 
  - 그리고 가격조회에서 검색시작일자 및 종료일자를 설정하여 가격을 조회합니다. 

## URL 조립 방법 안내
$$ $$
* 공통으로 사용되는 부분(main)과 세부항목(품목조회 or 가격조회), 그리고 인증키로 조립합니다. 
  - 아래 url은 품목조회에 사용되는 예시입니다. 
  - http://data.insight.go.kr:8080/openapi/service/PriceItemList/getPriceItemList?serviceKey=인증키
  - 가격조회를 할 때는 "PriceItemList/getPriceItemList" 대신  
    "PriceInfo/getPriceInfo"를 사용합니다.

In [1]:
# 필요한 라이브러리를 불러옵니다. 
library(httr)
library(rvest)
library(dplyr)
library(stringr)

Loading required package: xml2

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



## 1. 품목조회
$$ $$
* 통계청에서 제공하고 있는 123종의 물가 정보 품목 리스트를 수집합니다. 
$$ $$
* 여기에서 수집된 품목코드를 이용하여 '가격조회'를 할 수 있습니다. 

In [2]:
# url 요소를 설정합니다. 
main <- "http://data.insight.go.kr:8080/openapi/service/"
sub1 <- "PriceItemList/getPriceItemList"

# 인증키를 입력합니다. 
api_key <- "발급받은 인증키를 이곳에 입력하세요"

In [3]:
# 실제 인증키를 할당합니다. 
api_key <- "e5rYGavudWWkTkvxiFH7tJlfSSDX9EczjpD1k2zvLcrt5qJjIAKkX5aNRYj8wqxapIYlJZ0XwQ4ZY%2BKNFn5TEQ%3D%3D"

In [4]:
# url을 조립합니다. 
url <- paste0(main, 
              sub1, 
              paste0("?ServiceKey=", api_key))

In [5]:
# url을 확인합니다.
url

[1] "http://data.insight.go.kr:8080/openapi/service/PriceItemList/getPriceItemList?ServiceKey=e5rYGavudWWkTkvxiFH7tJlfSSDX9EczjpD1k2zvLcrt5qJjIAKkX5aNRYj8wqxapIYlJZ0XwQ4ZY%2BKNFn5TEQ%3D%3D"

In [6]:
# 조립한 url로 요청합니다. 
resp <- GET(url)
resp$status_code

[1] 200

In [7]:
# resp 객체의 content를 텍스트로 추출하여 구조를 파악합니다. 
# 만약 encoding이 'EUC-KR'로 되어 있으면, 
# content() 인자로 endcoding="EUC-KR"을 추가합니다. 
xmlObj <- content(resp, as="text")
cat(xmlObj)

# response > body > items 하위 노드로 item이 반복되고 있습니다. 
# 총 123개 항목명과 항목코드가 제공되고 있습니다. 

<?xml version="1.0" encoding="UTF-8" standalone="yes" ?>
<response>
  <body>
    <items>
      <item>
        <rn>1</rn>
        <ic>A011010</ic>
        <in>쌀</in>
      </item>
      <item>
        <rn>2</rn>
        <ic>A011020</ic>
        <in>찹쌀</in>
      </item>
      <item>
        <rn>3</rn>
        <ic>A011040</ic>
        <in>콩</in>
      </item>
      <item>
        <rn>4</rn>
        <ic>A011050</ic>
        <in>땅콩</in>
      </item>
      <item>
        <rn>5</rn>
        <ic>A011070</ic>
        <in>밀가루</in>
      </item>
      <item>
        <rn>6</rn>
        <ic>A011080</ic>
        <in>국수</in>
      </item>
      <item>
        <rn>7</rn>
        <ic>A011090</ic>
        <in>라면</in>
      </item>
      <item>
        <rn>8</rn>
        <ic>A011100</ic>
        <in>당면</in>
      </item>
      <item>
        <rn>9</rn>
        <ic>A011120</ic>
        <in>씨리얼식품</in>
      </item>
      <item>
        <rn>10</rn>
        <ic>A011150</ic>
        <in>식빵</in>
      </item

In [8]:
# 이제 XML을 불러옵니다. 
xml <- read_xml(url)
xml

{xml_document}
<response>
[1] <body>\n  <items>\n    <item>\n      <rn>1</rn>\n      <ic>A011010</ic>\n ...

In [9]:
# 조회 건수(totalCount)를 확인합니다. 
xml %>% xml_nodes("body totalCount") %>% xml_text()

[1] "123"

In [10]:
# 품목 리스트를 수집합니다. 
items <- xml %>% xml_nodes("item")
items

{xml_nodeset (123)}
 [1] <item>\n  <rn>1</rn>\n  <ic>A011010</ic>\n  <in>쌀</in>\n</item>
 [2] <item>\n  <rn>2</rn>\n  <ic>A011020</ic>\n  <in>찹쌀</in>\n</item>
 [3] <item>\n  <rn>3</rn>\n  <ic>A011040</ic>\n  <in>콩</in>\n</item>
 [4] <item>\n  <rn>4</rn>\n  <ic>A011050</ic>\n  <in>땅콩</in>\n</item>
 [5] <item>\n  <rn>5</rn>\n  <ic>A011070</ic>\n  <in>밀가루</in>\n</item>
 [6] <item>\n  <rn>6</rn>\n  <ic>A011080</ic>\n  <in>국수</in>\n</item>
 [7] <item>\n  <rn>7</rn>\n  <ic>A011090</ic>\n  <in>라면</in>\n</item>
 [8] <item>\n  <rn>8</rn>\n  <ic>A011100</ic>\n  <in>당면</in>\n</item>
 [9] <item>\n  <rn>9</rn>\n  <ic>A011120</ic>\n  <in>씨리얼식품</in>\n</item>
[10] <item>\n  <rn>10</rn>\n  <ic>A011150</ic>\n  <in>식빵</in>\n</item>
[11] <item>\n  <rn>11</rn>\n  <ic>A011160</ic>\n  <in>빵</in>\n</item>
[12] <item>\n  <rn>12</rn>\n  <ic>A012050</ic>\n  <in>소시지</in>\n</item>
[13] <item>\n  <rn>13</rn>\n  <ic>A012060</ic>\n  <in>햄</in>\n</item>
[14] <item>\n  <rn>14</rn>\n  <ic>A012070</ic>\n  <in>육류통조림</in>\

In [11]:
# 품목명만 문자열 벡터로 추출해보겠습니다. 
items %>% xml_nodes("in") %>% xml_text()

[1] "쌀"             "찹쌀"           "콩"             "땅콩"          
  [5] "밀가루"         "국수"           "라면"           "당면"          
  [9] "씨리얼식품"     "식빵"           "빵"             "소시지"        
 [13] "햄"             "육류통조림"     "고등어"         "전복"          
 [17] "마른멸치"       "북어채"         "생선통조림"     "분유"          
 [21] "치즈"           "참기름"         "식용유"         "사과"          
 [25] "배"             "과실통조림"     "고구마"         "토마토"        
 [29] "고사리"         "단무지"         "김"             "미역"          
 [33] "껌"             "아이스크림"     "비스킷"         "스낵과자"      
 [37] "설탕"           "잼"             "고춧가루"       "생강"          
 [41] "소금"           "간장"           "된장"           "고추장"        
 [45] "카레"           "케찹"           "드레싱"         "혼합조미료"    
 [49] "스프"           "이유식"         "김치"           "냉동식품"      
 [53] "즉석식품"       "커피"           "커피크림"       "차"            
 [57] "차음료"         "과일주스"       "생수"           "기능성음료"    
 [61] "탄산음료"       "혼합음료"       "남자정장"       "남자외투"      
 [65] "남자상의"       "남자하의"       "남자내의"       "여자정장"      
 [69] "여자외투"       "원피스"         "여자상의"       "여자하의"      
 [73] "여자내의"       "점퍼"           "티셔츠"         "스웨터"        
 [77] "청바지"         "운동복"         "등산복"         "아동복"        
 [81] "유아복"         "아동화"         "남자구두"       "여자구두"      
 [85] "운동화"         "실내화"         "세탁세제"       "섬유유연제"    
 [89] "전구"           "부엌용세제"     "청소용세제"     "살충제"        
 [93] "가정용비닐용품" "키친타월"       "방향제"         "습기제거제"    
 [97] "프린터"         "컴퓨터소모품"   "기록매체"       "공책"          
[101] "스케치북"       "복사용지"       "색메모지"       "필기구"        
[105] "회화용구"       "면도기"         "헤어드라이어"   "칫솔"          
[109] "치약"           "화장비누"       "샴푸"           "바디워시"      
[113] "화장지"         "화장수"         "로션"           "썬크림"        
[117] "영양크림"       "클린징크림"     "파운데이션"     "페이스파우더"  
[121] "립스틱"         "모발염색약"     "구강세정제"

In [12]:
# 위와 같은 방법으로 텍스트를 추출해주는 나만의 함수를 하나 만듭니다. 
getXmlText <- function(x, var) {
    result <- x %>% xml_node(var) %>% xml_text()
    return(result)
}

In [13]:
# 나만의 함수를 테스트합니다. 
itemNm <- getXmlText(items, "in")
itemNm

[1] "쌀"             "찹쌀"           "콩"             "땅콩"          
  [5] "밀가루"         "국수"           "라면"           "당면"          
  [9] "씨리얼식품"     "식빵"           "빵"             "소시지"        
 [13] "햄"             "육류통조림"     "고등어"         "전복"          
 [17] "마른멸치"       "북어채"         "생선통조림"     "분유"          
 [21] "치즈"           "참기름"         "식용유"         "사과"          
 [25] "배"             "과실통조림"     "고구마"         "토마토"        
 [29] "고사리"         "단무지"         "김"             "미역"          
 [33] "껌"             "아이스크림"     "비스킷"         "스낵과자"      
 [37] "설탕"           "잼"             "고춧가루"       "생강"          
 [41] "소금"           "간장"           "된장"           "고추장"        
 [45] "카레"           "케찹"           "드레싱"         "혼합조미료"    
 [49] "스프"           "이유식"         "김치"           "냉동식품"      
 [53] "즉석식품"       "커피"           "커피크림"       "차"            
 [57] "차음료"         "과일주스"       "생수"           "기능성음료"    
 [61] "탄산음료"       "혼합음료"       "남자정장"       "남자외투"      
 [65] "남자상의"       "남자하의"       "남자내의"       "여자정장"      
 [69] "여자외투"       "원피스"         "여자상의"       "여자하의"      
 [73] "여자내의"       "점퍼"           "티셔츠"         "스웨터"        
 [77] "청바지"         "운동복"         "등산복"         "아동복"        
 [81] "유아복"         "아동화"         "남자구두"       "여자구두"      
 [85] "운동화"         "실내화"         "세탁세제"       "섬유유연제"    
 [89] "전구"           "부엌용세제"     "청소용세제"     "살충제"        
 [93] "가정용비닐용품" "키친타월"       "방향제"         "습기제거제"    
 [97] "프린터"         "컴퓨터소모품"   "기록매체"       "공책"          
[101] "스케치북"       "복사용지"       "색메모지"       "필기구"        
[105] "회화용구"       "면도기"         "헤어드라이어"   "칫솔"          
[109] "치약"           "화장비누"       "샴푸"           "바디워시"      
[113] "화장지"         "화장수"         "로션"           "썬크림"        
[117] "영양크림"       "클린징크림"     "파운데이션"     "페이스파우더"  
[121] "립스틱"         "모발염색약"     "구강세정제"

In [14]:
# 이제 필요한 필요한 컬럼들로 구성된 데이터프레임을 생성해보겠습니다. 
itemList <- data.frame(itemCd = getXmlText(items, "ic"),
                       itemNm = getXmlText(items, "in"))

head(itemList)

itemCd,itemNm
A011010,쌀
A011020,찹쌀
A011040,콩
A011050,땅콩
A011070,밀가루
A011080,국수


## 2. 가격조회
$$ $$
* 앞에서 수집한 품목코드를 이용하여 각 품목별 온라인 가격을 수집합니다. 
$$ $$
* 가격조회 시, URL에 다음과 같은 조건을 추가합니다. 
  - 품목코드(itemCode) : 품목조회에서 수집한 코드
  - 검색시작일자(startDate), 검색종료일자(endDate) : 각각 8자리 날짜(YYYYMMDD)
  - 페이지 수(pageNo) : 생략하면 전체목록을 응답합니다.
  - 페이지당 항목 개수(numOfRows) : 생략하면 전체목록을 응답합니다.

In [15]:
# url 요소를 설정합니다. 
main <- "http://data.insight.go.kr:8080/openapi/service/"
sub2 <- "PriceInfo/getPriceInfo"

In [16]:
# 쌀 가격을 조회해보겠습니다.
itemCode <- itemList$itemCd[itemList$itemNm=="쌀"]
itemCode

[1] "A011010"

In [17]:
# 검색시작일자 및 검색종료일자를 설정합니다.
# 두 기간은 최대 30일로 제한됩니다. 
# 페이지번호 없이 url 요청 시, 최대 2일까지 권장한다고 합니다. 
# 이번 예제에서는 같은 날짜로 정해보겠습니다. 
strDate <- endDate <- "20170927"

In [18]:
# url을 조립합니다. 
# 전체 목록을 조회하기 위해 pageNo와 numOfRows는 설정하지 않습니다. 
url <- paste0(main, 
              sub2, 
              paste0("?ServiceKey=", api_key),
              paste0("&itemCode=", itemCode), 
              paste0("&startDate=", strDate),
              paste0("&endDate=", endDate))

In [19]:
# 조립한 url로 요청합니다. 
resp <- GET(url)
resp$status_code

[1] 200

In [20]:
# resp 객체의 content를 텍스트로 추출하여 구조를 파악합니다. 
xmlObj <- content(resp, as="text")
cat(xmlObj)

<?xml version="1.0" encoding="UTF-8" standalone="yes" ?>
<response>
  <body>
    <ic>A011010</ic>
    <in>쌀</in>
    <items>
      <item>
        <pi>998298630</pi>
        <pn>오곡밥도 일반밥도 간편히 즐기기</pn>
        <sp>3800.0</sp>
        <dp></dp>
        <bp></bp>
        <sd>2017-09-27</sd>
      </item>
      <item>
        <pi>998298629</pi>
        <pn>간편 즉석밥 모음</pn>
        <sp>3800.0</sp>
        <dp></dp>
        <bp></bp>
        <sd>2017-09-27</sd>
      </item>
      <item>
        <pi>997710710</pi>
        <pn>해빗 무농약 추청미(4KG)</pn>
        <sp>11480.0</sp>
        <dp></dp>
        <bp></bp>
        <sd>2017-09-27</sd>
      </item>
      <item>
        <pi>997710682</pi>
        <pn>해빗 무농약 추청미(2KG)</pn>
        <sp>5980.0</sp>
        <dp></dp>
        <bp></bp>
        <sd>2017-09-27</sd>
      </item>
      <item>
        <pi>992083463</pi>
        <pn>전국 쌀 20kg 골라담기(경기미/전라미/충청미/강원미) 16종</pn>
        <sp>47900.0</sp>
        <dp></dp>
        <bp></bp>
        <sd>2017-09-27</

In [21]:
# 이제 XML을 불러옵니다. 
xml <- read_xml(url)
xml

# 품목 리스트를 수집합니다. 
items <- xml %>% xml_nodes("item")

{xml_document}
<response>
[1] <body>\n  <ic>A011010</ic>\n  <in>쌀</in>\n  <items>\n    <item>\n      <p ...

In [22]:
# 품목명과 품목코드를 할당합니다. 
itemCode <- xml %>% xml_node("body ic") %>% xml_text
itemCode 

itemName <- xml %>% xml_node("body in") %>% xml_text
itemName

[1] "A011010"

[1] "쌀"

In [23]:
# 이제 필요한 필요한 컬럼들로 구성된 데이터프레임을 생성해보겠습니다. 
priceInfo <- data.frame(itemCd = itemCode, 
                        itemNm = itemName, 
                        prodId = getXmlText(items, "pi"),
                        prodNm = getXmlText(items, "pn"),
                        sPrice = getXmlText(items, "sp"),
                        dPrice = getXmlText(items, "dp"),
                        bPrice = getXmlText(items, "bp"),
                        piDate = getXmlText(items, "sd"))

head(priceInfo)

itemCd,itemNm,prodId,prodNm,sPrice,dPrice,bPrice,piDate
A011010,쌀,998298630,오곡밥도 일반밥도 간편히 즐기기,3800.0,,,2017-09-27
A011010,쌀,998298629,간편 즉석밥 모음,3800.0,,,2017-09-27
A011010,쌀,997710710,해빗 무농약 추청미(4KG),11480.0,,,2017-09-27
A011010,쌀,997710682,해빗 무농약 추청미(2KG),5980.0,,,2017-09-27
A011010,쌀,992083463,전국 쌀 20kg 골라담기(경기미/전라미/충청미/강원미) 16종,47900.0,,,2017-09-27
A011010,쌀,992081512,전국 쌀 10kg 골라담기(경기미/전라미/충청미/강원미) 14종,27900.0,,,2017-09-27


'data.frame':	669 obs. of  8 variables:
 $ itemCd: chr  "A011010" "A011010" "A011010" "A011010" ...
 $ itemNm: chr  "쌀" "쌀" "쌀" "쌀" ...
 $ prodId: chr  "998298630" "998298629" "997710710" "997710682" ...
 $ prodNm: chr  "오곡밥도 일반밥도 간편히 즐기기" "간편 즉석밥 모음" "해빗 무농약 추청미(4KG)" "해빗 무농약 추청미(2KG)" ...
 $ sPrice: chr  "3800.0" "3800.0" "11480.0" "5980.0" ...
 $ dPrice: chr  "" "" "" "" ...
 $ bPrice: chr  "" "" "" "" ...
 $ piDate: chr  "2017-09-27" "2017-09-27" "2017-09-27" "2017-09-27" ...


## 정리하며...
$$ $$
* 123종의 품목에 대해 일별로 수집을 하면 날짜별 전체 가격 정보를 모을 수 있습니다. 
  - 앞서 정리한 품목 리스트를 이용하여 순환 코드를 직접 짜보시기 바랍니다.
$$ $$
* 이번 예제에서 살펴본 바와 같이 "쌀" 품목에 대해서만 669개의 상품이 포함되어 있습니다. 
  - 각 상품명에서 알 수 있듯이, 같은 브랜드의 중량별로 금액이 다릅니다. 
  - 각 상품별 판매량을 알 수 있다면 해당 시장의 크기와 가중평균된 가격을 추정할 수 있을 것입니다. 

# End of Document